# PokerIQ Tutorial
_This tutorial runs through the basic usage of PokerIQ._

## Installation
First, install the library either through `pip` or source.

Then, import the six required classes. We are ready to go!

In [11]:
!pip install git+https://github.com/sumaddury/pokeriq.git

from pokeriq import Card, Hand, Deck, Player, Simulation, EquitySolver

  Cloning https://github.com/sumaddury/pokeriq.git to /private/var/folders/kl/wldt1h0141nbgk6x5ch6fs7h0000gn/T/pip-req-build-70zvszaw
  Running command git clone --filter=blob:none --quiet https://github.com/sumaddury/pokeriq.git /private/var/folders/kl/wldt1h0141nbgk6x5ch6fs7h0000gn/T/pip-req-build-70zvszaw
  Resolved https://github.com/sumaddury/pokeriq.git to commit de0ac931a93940d4fecf4e06ec292114ce262c1a
  Preparing metadata (setup.py) ... done


## The `Card` class

The `Card` class encapsulates the playing card, with individual instances containing information for individual cards.\
Attributes:
- `suit`: The card's suit (eg. 's' for spade)
- `rank`: The card's rank (eg. 7 for 7 or 12 for Q)
    
Essential static functions:
- `sequenceToString(sequence)`: Converts list of `Card` objects to a string message
- `generate(card)`: Returns a `Card` object corresponding to the input string representation. Format is RankSuit (eg. 'Ts', '9h')
- `generateSet(sequence)`: Converts collection of card string representations, to the corresponding list of `Card` objects
- `generateSetofSets(set)`: Converts a collection of sequences as indicated above, to the corresponding list of lists of `Card` objects

In [42]:
queenOfSpadesOne = Card('s', 12)
queenOfSpadesTwo = Card.generate('Qs')
print(queenOfSpadesOne.toString(), queenOfSpadesTwo.toString())

♠Q ♠Q


In [44]:
pocketEightsOne = [Card('h', 8), Card('s', 8)]
pocketEightsTwo = Card.generateSet(['8h','8s'])
print(Card.sequenceToString(pocketEightsOne))
print(Card.sequenceToString(pocketEightsTwo))

 ♥8 ♠8
 ♥8 ♠8


We will cover `generateSetofSets()` later, when we explore ranges. These generators are intended to provide less cumbersome mechanisms to create `Card` data.

`Card` also has many instance functions which you can play around with. (Apart from the constructor, however, they are mostly for internal use.)

## The `EquitySolver` class

The `EquitySolver` class is the main product of this library. It provides functionality for calculating hand and range equity.\
Essential instance methods:
- `addPlayers(amnt)`: Adds up to 10 total players to the solver (starts with 0). Default value is 1
- `removePlayers(amnt)`: Removes any number of players from the solver, prioritizing MOST RECENT. Default value is 1
- `defineHole(id, hole)`: Defines hole cards for a specific player, where `id` is an integer corresponding to the number of the player. `hole` must be a list of exactly two unequal `Card` objects
- `clearHole(id)`: Resets hole cards for a specific player to empty based on `id`
- `defineBoard(board)`: Defines the starting board used by the solver if desired. If this function is not used, calculation will start from preflop. Input `board` must be a list of `Card` objects from length 3-5
- `calculateHandEquity(trials)`: After players, boards, and decks have been added and any hole cards defined, this function calculates hand equity for each player from the point provided (NO BOARD: preflop; 3 CARDS: flop; etc.). If hole cards are not provided for one or more player, each trial will assign a random hand from the deck to each of these players. Returns a dictionary of equities, which will also be stored internally. `trials` can be specified if desired, (1,000-10,000 trials generally provides sufficient accuracy). Default value is 1,000
- `toString()`: Will return a string message of the solver state, with hand equities if `calculateHandEquity()` has been called

*Other instance methods can be found in spec.
Essential static methods:
- `generateRange(range)`: Generates a collection of all possible hole card pairs based on a range. Essential for generating large ranges without specifying every suit combination. Input `range` should be a list of generic hand string representations, with pocket pairs indicated by the two cards and unpaired hands indicated by the two cards followed by 'o' for offsuit and 's' for suited (eg. '88', 'TT', 'AA', 'AKo', 'T9s', '72o', etc.). Ex. input: `['AA', 'KK', 'QQ', 'AKs', 'AKo']` will return a list of 34 `Card` pairs, corresponding to each possible suit combination of these hand generics. This function is an alternative to `Card.generateSetofSets()`
- `calculateRangeEquity(*args, trials, customDeck, customBoard)`: Calculates the AVERAGE equity for each of any number of ranges (inputted via `*args`). Uses a Cartesian product to enumerate over every possible combination that can be made between the ranges (4 hand range vs 6 hand range vs 16 hand range means 4*6*16=384 enumerations), running each for a specified number of trials (default is 1000). A deck and/or board input can also be provided, and equity calculation will begin from there (see `calculateHandEquity` for specifics). Returns a tuple with the equity dictionary and a string message




In [27]:
solver = EquitySolver()
solver.addPlayers(3)
solver.defineHole(1, Card.generateSet({'Ah','5h'}))
solver.defineHole(2, Card.generateSet({'Tc','Ts'}))
solver.defineBoard(Card.generateSet(['6h','7h','9s']))
solver.calculateHandEquity(trials=1000)
print(solver.toString())

____________________
Board Cards: 
 ♥6 ♥7 ♠9
____________________
Player 1 |  ♥5 ♥A | 45.400000000000034%
Player 2 |  ♠T ♣T | 41.30000000000003%
Player 3 |  | 15.10000000000001%
Chop | 0%
____________________



In [46]:
fiveBetRangeOne = Card.generateSetofSets({('As','Ah'), ('As','Ad'), ('As','Ac'), ('Ah','Ad'), ('Ah','Ac'), ('Ad','Ac'), 
                                    ('Ks','Kh'), ('Ks','Kd'), ('Ks','Kc'), ('Kh','Kd'), ('Kh','Kc'), ('Kd','Kc'), 
                                    ('Qs','Qh'), ('Qs','Qd'), ('Qs','Qc'), ('Qh','Qd'), ('Qh','Qc'), ('Qd','Qc'), 
                                    ('As','Ks'), ('Ah','Kh'), ('Ad','Kd'), ('Ac','Kc'), ('As','Kh'), ('As','Kd'), 
                                    ('As','Kc'), ('Ah','Ks'), ('Ah','Kd'), ('Ah','Kc'), ('Ad','Ks'), ('Ad','Kh'), 
                                    ('Ad','Kc'), ('Ac','Ks'), ('Ac','Kh'), ('Ac','Kd')})
fiveBetRangeTwo = EquitySolver.generateRange({'AA', 'KK', 'QQ', 'AKs', 'AKo'})
toStringListOne = [Card.sequenceToString(hand) for hand in fiveBetRangeOne]
toStringListTwo = [Card.sequenceToString(hand) for hand in fiveBetRangeTwo]
print(toStringListOne)
print(toStringListTwo)

[' ♠K ♦K', ' ♦A ♥K', ' ♦A ♣A', ' ♠A ♥A', ' ♦A ♣K', ' ♦K ♣K', ' ♥A ♠K', ' ♥A ♦K', ' ♠A ♦A', ' ♥K ♣K', ' ♠Q ♥Q', ' ♥Q ♣Q', ' ♣A ♠K', ' ♣A ♦K', ' ♠A ♥K', ' ♠A ♣A', ' ♠A ♣K', ' ♦A ♠K', ' ♠K ♥K', ' ♦A ♦K', ' ♠Q ♦Q', ' ♥A ♦A', ' ♠K ♣K', ' ♥K ♦K', ' ♥A ♥K', ' ♥A ♣A', ' ♥A ♣K', ' ♠A ♠K', ' ♣A ♥K', ' ♠A ♦K', ' ♦Q ♣Q', ' ♥Q ♦Q', ' ♣A ♣K', ' ♠Q ♣Q']
[' ♠A ♥K', ' ♠A ♦K', ' ♠A ♣K', ' ♥A ♠K', ' ♥A ♦K', ' ♥A ♣K', ' ♦A ♠K', ' ♦A ♥K', ' ♦A ♣K', ' ♣A ♠K', ' ♣A ♥K', ' ♣A ♦K', ' ♠A ♠K', ' ♥A ♥K', ' ♦A ♦K', ' ♣A ♣K', ' ♠K ♥K', ' ♠K ♦K', ' ♠K ♣K', ' ♥K ♦K', ' ♥K ♣K', ' ♦K ♣K', ' ♠Q ♥Q', ' ♠Q ♦Q', ' ♠Q ♣Q', ' ♥Q ♦Q', ' ♥Q ♣Q', ' ♦Q ♣Q', ' ♠A ♥A', ' ♠A ♦A', ' ♠A ♣A', ' ♥A ♦A', ' ♥A ♣A', ' ♦A ♣A']


In [50]:
myRange = EquitySolver.generateRange({'AA', 'KK'})
equities, message = EquitySolver.calculateRangeEquity(myRange, fiveBetRangeTwo, trials=3000)
print(message)

____________________
BoardCards: 

____________________
Range 1 | 59.25890522875673%
Range 2 | 21.538888888888692%
CHOP | 19.202205882352214%
____________________



In [51]:
myHand = [Card.generateSet(('9c','9s'))]
uncappedThreeBetRange = EquitySolver.generateRange({'AA','KK','QQ','AKs','AKo','JJ','TT','99',
                                                    'AQs','AQo','AJs','AJo','ATs','ATo','KQs','KQo'})
board = Card.generateSet(['Ac','9h','2h'])
equities, message = EquitySolver.calculateRangeEquity(myHand, uncappedThreeBetRange, customBoard=board)
print(message)

____________________
BoardCards: 
 ♣A ♥9 ♥2
____________________
Range 1 | 84.08879310344834%
Range 2 | 10.870689655172425%
CHOP | 5.040517241379313%
____________________



## The `Hand`, `Deck`, and `Player` classes

The `Hand` class encapsulates a MADE hand, providing functionality for hand assessment and comparison.\
_Attributes:_
- _`hand`: The object's hand type (eg. 'STRAIGHT FLUSH', 'TWO PAIR', etc.)_
- _`strength`: An array corresponding to how this specific object compares within hand types (eg. K high flush vs 6 high flush)_

The `Deck` class encapsulates a card data structure with a variety of dealing functions.\
_Attributes:_
- _`cards`: List of Card objects corresponding to the `Card` data_

The `Player` class represents individual players with functionality for winner assessment, dealing, and betting.\
_Attributes:_
- _`name`: Instance' string name_
- _`stack`: Numerical chip stack_
- _`hole`: List of Card objects containing the player's hole cards_
- _`currentBet`: The last bet amount placed by the player_

**`Hand` is strictly for internal use. Its instance and static methods do not check for preconditions and should not be used by users.\
`Deck` and `Player` are also mostly for internal use, but are user-approved. Read the source code specs for specific information on how to use them for custom simulations.**

## The `Simulation` class

The `Simulation` class provides functionality for all types of simulations.\
_Attributes:_
- _`flop`: List of length 3 of `Card` objects corresponding to the flop_
- _`turn`: List of length 1 of `Card` objects corresponding to the turn_
- _`river`: List of length 1 of `Card` objects corresponding to the river_
- _`board`: Concatenation of `flop`, `turn`, and `river`_
- _`players`: List of `Player` objects participating in the trial_
- _`winners`: List of `Player` object(s) who won the hand_
- _`highHand`: String representation of the made hand that won (eg. 'FULL HOUSE')_

The `Simulation` class is used by `EquitySolver` for each trial, but can also be used directly for custom simulations via `runSim()`. This method assumes that any `customDeck` inputs are up-to-date (board and player cards have been removed). See the source code spec for specific information.

This was a high level tutorial that shows the most essential features of this micro-library. There is plenty of other customizable functionality provided, such as in the later three classes. All of the source code is clearly commented and is worth reading!